# 6장 데이터 로딩과 저장, 파일 형식

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)
plt.rcParams["font.family"] = 'Malgun Gothic'

# 1. csv 텍스트 데이터
## 1.1 csv파일 읽기
* 판다스에서 CSV 파일을 읽기: pd.read_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


    1) header(컬럼명)가 있는 파일

In [8]:
!cat examples/ex1.csv

df = pd.read_csv("examples/ex1.csv", encoding= "utf-8") # csv file -> DataFrame으로 반환
type(df)
df.head(2)

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world


    2) header가 없는 파일 : 컬럼을 지정
        * names 인수 - 열 이름 리스트
        * header 인수 - 헤더가 없을 경우 None으로 지정

In [12]:
!cat examples/ex2.csv

names = ["a", "b", "c", "d", "message"]
data = pd.read_csv("examples/ex2.csv", names=names) #names 인수 - 데이터프레임의 컬럼명
data

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    3) 계층적 색인 지정하기
        * index_col 인수 - 색인으로 사용할 열 번호나 이름, 계층적 색인을 지정할 경우 리스트

In [17]:
!cat examples/csv_mindex.csv
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

    4) 여러 개의 공백(spacebar)으로 필드를 구분한 파일 : txt포맷 
        * sep 인수 - 필드을 구분하기 위해 사용할 정규 표현식

In [20]:
!cat examples/ex3.txt
result = pd.read_csv("examples/ex3.txt", sep ="/s+") # 정규표현식 : " /s+ "
result

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


C:\Users\user\AppData\Local\Temp\ipykernel_7704\3931617539.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  result = pd.read_csv("examples/ex3.txt", sep ="/s+") # 정규표현식 : " /s+ "


,A B C
0,aaa -0.264438 -1.026059 -0.619500
1,bbb 0.927272 0.302904 -0.032399
2,ccc -0.264273 -0.386314 -0.217601
3,ddd -0.871858 -0.348382 1.100491


    5) 주석을 갖고 있는 파일
        * skiprows 인수 - 파일의 시작부터 무시할 행 수 또는 무시할 행 번호가 담긴 리스트

In [21]:
!cat examples/ex4.csv
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    6) 누락된 값이 있는 파일 : 비어 있는 문자열, NA, NULL을 NaN으로 출력 
        * na_values 인수 - NA 값으로 처리할 값들의 목록

In [26]:
!cat examples/ex5.csv
# na_values 인수를 사용하지 않은 경우
result = pd.read_csv("examples/ex5.csv")
result
# 1) 누락된 값을 확인 : isna(), isnull(), notna(), notnull()
pd.isna(result)

# 2) na_values -> 컬럼별 문자열 집합을 받아서 누락된 값 처리
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
result = pd.read_csv("examples/ex5.csv", na_values=sentinels)
print(result)
result.isna()

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo
  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,True,False,False,True,False,False
2,False,False,False,False,False,True


    7) 텍스트 파일 조금씩 
        * nrows 인수 - 파일의 첫 일부만 읽어올 때 처음 몇 줄을 읽을 것인지 지정 
        * chunksize 인수 - TextParser 객체에서 사용할 한 번에 익을 파일의 크기

In [29]:
# pd.options.display.max_rows = 10
# !cat examples/ex6.csv
# result = pd.read_csv("examples/ex6.csv")
# result
result = pd.read_csv("examples/ex6.csv", nrows=5)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


### [예제]  ex6.csv 파일을 순회하면서 "key"열의 값을 세기

In [52]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
chunker
# type(chunker)
#1) 리스트의 시리즈 생성 - 청크 단위로 key"열의 값 저장
tot = pd.Series([], dtype='int64')
type(tot)
#2) 반복 처리 

for piece in chunker:
    #print(type(piece["key"]))
    #print(piece["key"].value_counts())
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot
# tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

In [46]:
# note
from pandas import Series
tot = pd.Series([], dtype='int64')
type(tot)

for piece in range(3):
    tot = tot.add(Series([1,2,3,]), fill_value = 10) # tot + Series([1,2,3])
tot

0    13.0
1    16.0
2    19.0
dtype: float64

## 1.2. csv 텍스트 파일 저장하기
* 판다스에서 CSV 파일을 저장하기: df.to_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


In [77]:
!cat examples/ex5.csv
# # 1) 준비 : DataFrame
data = pd.read_csv("examples/ex5.csv")  # -> DataFrame 반환
# # 2. 조작 : DataFrame에서 인덱싱해서 필요한 데이터 추출->DataFrame
data1 = data[['a','b','d']]
print(data1)
# 3. data1 -> csv 파일로 저징
data1.to_csv("examples/out.csv")


something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo
   a   b   d
0  1   2   4
1  5   6   8
2  9  10  12


In [81]:
data1.to_csv("examples/out.csv", sep="|") # 구분자 사용
data1.to_csv("examples/out.csv", na_rep = "NULL") #누락된 값을 원하는 값으로 지정
# data.to_csv("examples/out.csv", index=False, header=False)
data1.to_csv("examples/out.csv", index=False, columns=["a", "b", "d"])




# 2. JSON  데이터
* JSON(JavaScript Object Notation)은 텍스트 데이터를 저장하고 전송하기 위한 일반적인 형식
* 주로 웹에서 데이터를 전송하거나 저장하는 데 사용
* JSON 파일은 일반적으로 텍스트 형식으로 작성되며, 사람과 기계 모두 이해하기 쉽다.
* JSON 파일은 키-값 쌍의 모음으로 구성
  * 키-값 쌍은 객체(object)라고도 하며, 중괄호 {} 로 묶는다.
  * 배열(array)은 대괄호 [] 안에 값의 목록을 나열하여 정의.

## 2.1 JSON 데이터 읽기/저장
    * pandas의 to_json() -> 데이터프레임을 JSON 파일로 저
    * read_json() -> JSON 파일에서 데이터프레임을 읽어오기

### [예제]  pandas의 to_json() 및 read_json() 메서드를 사용하여 데이터프레임을 JSON 파일로 저장하고 읽기

In [83]:
# 1. pandas 라이브러리 임포트

import pandas as pd

# 2. JSON 파일로 저장할 데이터프레임 생성
data = {
    "name": ["John", "Alice", "Bob"],
    "age": [30, 25, 35],
    "city": ["New York", "Los Angeles", "Chicago"]
}
df = pd.DataFrame(data)

# 3. 데이터프레임을 JSON 파일로 저장
df.to_json("examples/data.json")

# JSON 파일을 읽어와 데이터프레임으로 변환
df_read = pd.read_json("examples/data.json")

print("Original DataFrame:")
print(df)

print("\nDataFrame from JSON file:")
print(df_read)


Original DataFrame:
    name  age         city
0   John   30     New York
1  Alice   25  Los Angeles
2    Bob   35      Chicago

DataFrame from JSON file:
    name  age         city
0   John   30     New York
1  Alice   25  Los Angeles
2    Bob   35      Chicago


# 3. HTML 데이터
## 3.1 웹 스크래핑 - read_html()

* read_html() - html 문서 내 테이블 데이터 읽어 테이블 구조의 모든 데이터를 데이터프레임으로 저장해서 반환
    * https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/


# 개발자 도구 사용 방법
* #s_content > div.section > ul > li:nth-child(1) > dl > dt > a

#### [예제] 미국 예금 보험 공사에서 부도 은행을 보여주는 HTML 문서 읽어오기

In [89]:
import pandas as pd
from pandas import Series, DataFrame

tables = pd.read_html("https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/")
type(tables)
len(tables)
failures = tables[0]
failures
type(failures)
failures.head()

,Bank NameBank,CityCity,StateSt,CertCert,Acquiring InstitutionAI,Closing DateClosing,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543
3,Signature Bank,New York,NY,57053,"Flagstar Bank, N.A.","March 12, 2023",10540
4,Silicon Valley Bank,Santa Clara,CA,24735,First–Citizens Bank & Trust Company,"March 10, 2023",10539


In [93]:
# 데이터프레임 가공
# 1) 컬럼명 수정하기
failures.columns
failures.columns = ['BankNameBank', 'CityCity', 'StateSt', 'CertCert',
        'AcquiringInstitutionAI', 'ClosingDate', 'FundFund']
failures.head(3)

,BankNameBank,CityCity,StateSt,CertCert,AcquiringInstitutionAI,ClosingDate,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543


In [100]:
# 2) 데이터 변환 : 문자열 object -> datetime
close_timestamps = pd.to_datetime(failures["ClosingDate"]) # 컬럼 인덱싱
close_timestamps                   
#3) 년도별 부도난 은행의 개수 추출
close_timestamps.dt.year.value_counts() # Series

ClosingDate
2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2019      4
2001      4
2007      3
2003      3
2000      2
Name: count, Length: 19, dtype: int64

#### [실습] : 네이버 주식 데이터 가져오기 - read_html()
* https://www.naver.com/ -> 증권 -> 국내증시 -> 시가총액
    * https://finance.naver.com/sise/sise_market_sum.naver

In [ ]:
!pip install html5lib  lxml

In [107]:
import pandas as pd

# 'euc-kr' 또는 'cp949'로 인코딩 설정하여 페이지 읽기 시도
df_list = pd.read_html("https://finance.naver.com/sise/sise_market_sum.naver", encoding = 'cp949')
type(df_list)
len(df_list)
type(df_list[1])
# # 데이터프레임 확인
for df in df_list:
     print("\n",df.head())


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")



      0      1          2        3          4       5
0  거래량   매수호가   거래대금(백만)  시가총액(억)    영업이익(억)  PER(배)
1   시가   매도호가      전일거래량  자산총계(억)    영업이익증가율  ROE(%)
2   고가  매수총잔량      외국인비율  부채총계(억)   당기순이익(억)  ROA(%)
3   저가  매도총잔량  상장주식수(천주)   매출액(억)   주당순이익(원)  PBR(배)
4  NaN    NaN        NaN   매출액증가율  보통주배당금(원)  유보율(%)

      N       종목명       현재가      전일비     등락률     액면가       시가총액      상장주식수  \
0  NaN       NaN       NaN      NaN     NaN     NaN        NaN        NaN   
1  1.0      삼성전자   84100.0    900.0  -1.06%   100.0  5020587.0  5969783.0   
2  2.0    SK하이닉스  179200.0   7100.0  -3.81%  5000.0  1304580.0   728002.0   
3  3.0  LG에너지솔루션  376000.0  17000.0  -4.33%   500.0   879840.0   234000.0   
4  4.0  삼성바이오로직스  812000.0   4000.0  -0.49%  2500.0   577933.0    71174.0   

   외국인비율         거래량    PER    ROE  토론실  
0    NaN         NaN    NaN    NaN  NaN  
1  55.57  30136601.0  39.47   4.15  NaN  
2  54.69   5082907.0 -14.32 -15.61  NaN  
3   4.97    364327.0  71.12   6.36  NaN  
4  11

In [113]:
# 데이터프레임 가공- 종목명에 nan이 있는 행은 제거
df = df_list[1] # 컬럼 인덱싱
df.head(10)
#1.boolean indexing
df['종목명'] # 컬럼 추출 -Series
df['종목명'].notnull() #2. 조건 색인
data = df[df['종목명'].notnull()] # 조건색인으로 인덱싱하기
data
data.head()
# 가공한 데이터프레임을 파일로 저장
data.to_csv("examples/Naver_kospi.csv")

## 3.2 웹 스크래핑 - requests

* 웹 페이지 정보 가져오기 - requests 패키지 이용
  * requests.get()
  * requests.get().status_code
  * requests.get().text


### [예제] 타겟 사이트의 json 구조로 저장돤 데이터 가져오기

In [114]:
!pip install requests

### [예제] 

In [131]:

#1. requests 모듈 임포트하기
import requests
import pandas as pd
#3. url 가져오기
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
#pd.read = read_html(url)
# 페이지를 가져오기
resp = requests.get(url)
type(resp)
resp.status_code #200
resp.text # 데이터 가져오기
type(resp.text)

# # 4. json 내용을 딕셔너리로 변환
data = resp.json() # json의 내용을 딕셔너리 형태로 변환한 객체을 반환
type(data)
data



{'message': "API rate limit exceeded for 58.233.133.18. (But here's the good news: Authenticated requests get a higher rate limit. Check out the documentation for more details.)",
 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}

### [문제] 네이버 블로그의 키워드 관련 검색 결과 텍스트 데이터 가져오기 - requests 모듈 사용시 한계점

In [136]:
import requests
url = https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC

params = {
    'pageNo' : 1,
    'rangeType' : 'ALL',
    'orderBy' : 'sim',
    'keyword' : '파이썬'
}
response = response.get('https://section.blog.naver.com/Search/Post.naver?', params=params)
# pd.read_html('https://section.blog.naver.com/Search/Post.naver?', params=params)

print(response.status_code) # HTTP GET 방식으로 URL 파라미터 값 전달
print(response.url) # status_code 는 응답코드
print(response.text) # text에는 HTML 코드

NameError: name 'response' is not defined

## 3.3 웹 스크래핑 - beautifulsoup
* 웹 페이지 정보 추출하기 - beautifulsoup 사용법 
  * soup.select_one()
  * soup.select()
  * get_text()
* 개발자 도구 사용

### [문제] 네이버 지식iN의 검색어 관련 검색 결과 제목 가져오기

In [38]:
import requests
from bs4 import BeautifulSoup

url = 'https://kin.naver.com/search/list.naver?query=%ED%8C%8C%EC%9D%B4%EC%8D%AC'

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    # html -> beautifulsoup 객체로 변환
    soup = BeautifulSoup(html, 'html.parser')

    # 제목 한개 선택하여 텍스트 가져오기
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    print(title.get_text())

    # 제목 여러 개 선택하여 텍스트 리스트 가져오기
    titles = soup.select('#s_content > div.section > ul > li:nth-child(n) > dl > dt > a')
    for title in titles:
        print(title.get_text())
else : 
    print(response.status_code)



비주얼스튜디오코드 파이썬 질문합니다,
비주얼스튜디오코드 파이썬 질문합니다,
맥북 파이썬 idel
파이썬 모듈 미리 불러오기
파이썬과 자바
IDLE 파이썬 단축키 알려주세요
파이썬강좌 독학할때 뭐보나요??
파이썬이 미래에 도움이 될까요?
파이썬강의 듣는데가 있나요?
파이썬 라이선스 분석
파이썬독학할 때 어떻게 공부를... 


## 3.4 웹 스크래핑 - Selenium을 이용하여 사이트 자동화하기
* selenium 사용법
  1. 시스템의 OS 및 Chrome 브라우저의 버전 확인
  2. Chrome 드라이버 다운로드 및 설치
  3. 크롬 드라이버을 프로젝트 폴더에 저장 - 크롬 드라이버의 설치 경로
     * from selenium import webdriver
     * driver = webdriver.Chrome()
* 동적으로 웹 페이지 정보 추출하기
     * driver.get("웹주소") 
  

### [예제]

In [220]:
!pip install selenium

In [41]:
from selenium import webdriver
# Chrome 브라우저 실행
driver = webdriver.Chrome()
# 웹페이지 열기
url = "httpss://www.google.com"
driver.get(url)

### [문제]


In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# Chrome 브라우저 실행
driver = webdriver.Chrome()

# Google 홈페이지 열기
url = "https://www.google.com"
driver.get(url)

# 페이지가 완전히 로드될 때까지 2초 대기
time.sleep(5)

try:
    # 검색 상자 요소 찾기
    search_box = driver.find_element(By.NAME, "q")

    # 검색어 입력
    search_query = "OpenAI"
    search_box.send_keys(search_query)

    # Enter 키를 눌러 검색 수행
    search_box.send_keys(Keys.RETURN)

    # 검색 결과가 로드될 때까지 5초 대기
    time.sleep(5)

    # 검색 결과 추출
    search_results = driver.find_elements(By.CSS_SELECTOR, "div.g")

    # 검색 결과의 제목과 URL 인쇄
    for result in search_results:
        title = result.find_element(By.CSS_SELECTOR, "h3").text
        url = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        print("제목:", title)
        print("URL:", url)
        print()

except Exception as e:
    print("오류가 발생했습니다:", e)

finally:
    # 브라우저 창 닫기
    driver.quit()


제목: OpenAI
URL: https://openai.com/

제목: OpenAI
URL: https://openai.com/

제목: OpenAI
URL: https://namu.wiki/w/OpenAI

제목: OpenAI
URL: https://en.wikipedia.org/wiki/OpenAI

제목: 오픈AI - 위키백과, 우리 모두의 백과사전
URL: https://ko.wikipedia.org/wiki/%EC%98%A4%ED%94%88AI

제목: OpenAI
URL: https://www.youtube.com/@OpenAI

제목: OpenAI
URL: https://twitter.com/OpenAI

제목: 설명
URL: https://www.google.com/search?q=OpenAI&sca_esv=31475c1a0531b981&source=hp&ei=Xm0LZpnQBb632roP0Pyk2Ao&iflsig=ANes7DEAAAAAZgt7bsUIBJHPilalMMgyZAgXLw1v16Ob&udm=&ved=0ahUKEwiZ98PsvaKFAxW-m1YBHVA-CasQ4dUDCA8&uact=5&oq=OpenAI&gs_lp=Egdnd3Mtd2l6IgZPcGVuQUlIMlAAWA9wAHgAkAEAmAEAoAEAqgEAuAEDyAEA-AEBmAIAoAIAmAMAkgcAoAcA&sclient=gws-wiz#

